In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Dict
import random
from torch.cuda.amp import autocast, GradScaler
import logging
from IPython.display import display, HTML
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Set random seeds for reproducibility
def set_seeds(seed: int = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seeds(42)

In [ ]:
# Configuration
CONFIG = {
    'input_size': 6,  # Number of input features
    'sequence_length': 10,  # Number of time steps to look back
    'batch_size': 32,
    'learning_rate': 0.001,
    'epochs': 100,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print(f"Using device: {CONFIG['device']}")

In [ ]:
class WindSpeedDataset(Dataset):
    def __init__(self, data: np.ndarray, sequence_length: int):
        self.data = torch.FloatTensor(data)
        self.sequence_length = sequence_length

    def __len__(self) -> int:
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        x = self.data[idx:idx + self.sequence_length, :-1]
        y = self.data[idx + self.sequence_length - 1, -1]
        return x, y

In [ ]:
def preprocess_data(data: pd.DataFrame) -> Tuple[np.ndarray, MinMaxScaler]:
    """
    Preprocess the data with both scaling and statistical normalization
    """
    # First apply MinMax scaling
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)

    # Apply statistical normalization
    mean = np.mean(scaled_data, axis=0)
    std = np.std(scaled_data, axis=0)
    normalized_data = (scaled_data - mean) / (std + 1e-8)

    return normalized_data, scaler

In [ ]:
# Load and preprocess data
data = pd.read_csv('data.csv')
display(data.head())
print("\nData shape:", data.shape)

In [ ]:
# Plot input features distribution
plt.figure(figsize=(15, 5))
data.boxplot()
plt.title('Distribution of Wind Speeds at Different Heights')
plt.ylabel('Wind Speed (m/s)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int, dropout: float):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Add batch normalization for input
        self.input_bn = nn.BatchNorm1d(input_size)

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
        )

        # Add batch normalization after LSTM
        self.hidden_bn = nn.BatchNorm1d(hidden_size)

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch_size = x.size(0)
        seq_len = x.size(1)

        # Apply input batch normalization
        x = x.view(-1, x.size(-1))
        x = self.input_bn(x)
        x = x.view(batch_size, seq_len, -1)

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        # Apply batch normalization on the output features
        out = out[:, -1, :]
        out = self.hidden_bn(out)

        out = self.fc(out)
        return out.squeeze()

In [ ]:
class SparrowSearch:
    def __init__(self, n_particles: int, max_iter: int, param_bounds: Dict):
        self.n_particles = n_particles
        self.max_iter = max_iter
        self.param_bounds = param_bounds
        self.best_solution = None
        self.best_fitness = float('inf')

    def initialize_population(self) -> List[Dict]:
        population = []
        for _ in range(self.n_particles):
            particle = {}
            for param, (low, high) in self.param_bounds.items():
                if isinstance(low, int) and isinstance(high, int):
                    particle[param] = random.randint(low, high)
                else:
                    particle[param] = random.uniform(low, high)
            population.append(particle)
        return population

    def update_position(self, particle: Dict, r2: float, alarm_value: float) -> Dict:
        new_particle = particle.copy()
        for param, (low, high) in self.param_bounds.items():
            if random.random() < alarm_value:
                if isinstance(low, int) and isinstance(high, int):
                    new_particle[param] = random.randint(low, high)
                else:
                    new_particle[param] = random.uniform(low, high)
            else:
                step = r2 * (self.best_solution[param] - particle[param])
                new_value = particle[param] + step
                if isinstance(low, int) and isinstance(high, int):
                    new_value = int(round(new_value))
                new_particle[param] = max(low, min(high, new_value))
        return new_particle

    def optimize(self, fitness_func) -> Tuple[Dict, float]:
        population = self.initialize_population()
        history = []

        # Add tqdm progress bar
        pbar = tqdm(range(self.max_iter), desc='Sparrow Search Progress')
        
        for iteration in pbar:
            alarm_value = 0.5 - (0.5 * iteration / self.max_iter)

            for i, particle in enumerate(population):
                fitness = fitness_func(particle)

                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = particle.copy()
                    # Update progress bar description with current best fitness
                    pbar.set_postfix({'Best Fitness': f'{self.best_fitness:.6f}'})

            r2 = random.random()
            population = [self.update_position(p, r2, alarm_value) for p in population]

            history.append(self.best_fitness)
            logger.info(f"Iteration {iteration + 1}/{self.max_iter}, Best fitness: {self.best_fitness:.6f}")

        # Plot optimization history
        plt.figure(figsize=(10, 5))
        plt.plot(history)
        plt.title('Sparrow Search Optimization History')
        plt.xlabel('Iteration')
        plt.ylabel('Best Fitness')
        plt.show()

        return self.best_solution, self.best_fitness

In [ ]:
def create_data_loaders(
    scaled_data: np.ndarray,
    sequence_length: int,
    batch_size: int,
    train_ratio: float = 0.8
) -> Tuple[DataLoader, DataLoader]:
    train_size = int(len(scaled_data) * train_ratio)
    train_data = scaled_data[:train_size]
    val_data = scaled_data[train_size:]

    train_dataset = WindSpeedDataset(train_data, sequence_length)
    val_dataset = WindSpeedDataset(val_data, sequence_length)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    return train_loader, val_loader

In [ ]:
def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    device: str,
    epochs: int
) -> Dict[str, List[float]]:
    model = model.to(device)
    scaler = GradScaler()
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(epochs):
        model.train()
        train_losses = []

        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            optimizer.zero_grad()

            with autocast():
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_losses.append(loss.item())

        model.eval()
        val_losses = []
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                val_loss = criterion(outputs, batch_y)
                val_losses.append(val_loss.item())

        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)

        if (epoch + 1) % 10 == 0:
            logger.info(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    return history

In [ ]:
# Prepare data
scaled_data, scaler = preprocess_data(data)
train_loader, val_loader = create_data_loaders(
    scaled_data,
    CONFIG['sequence_length'],
    CONFIG['batch_size']
)

# Define parameter bounds for Sparrow Search
param_bounds = {
    'hidden_size': (32, 256),
    'num_layers': (1, 4),
    'dropout': (0.0, 0.5),
    'learning_rate': (0.0001, 0.01)
}

# Initialize Sparrow Search
sparrow = SparrowSearch(n_particles=20, max_iter=30, param_bounds=param_bounds)

In [ ]:
# Fitness function
def fitness_function(params):
    model = LSTM(
        input_size=CONFIG['input_size'],
        hidden_size=int(params['hidden_size']),
        num_layers=int(params['num_layers']),
        dropout=params['dropout']
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
    criterion = nn.MSELoss()

    history = train_model(
        model, train_loader, val_loader,
        criterion, optimizer,
        CONFIG['device'], CONFIG['epochs']
    )

    return np.mean(history['val_loss'])

# Run optimization
best_params, best_fitness = sparrow.optimize(fitness_function)
print("\nBest parameters found:", best_params)

In [ ]:
def evaluate_model(
    model: nn.Module,
    data_loader: DataLoader,
    scaler: MinMaxScaler,
    device: str
) -> Dict[str, float]:
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_x, batch_y in data_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)

            predictions.extend(outputs.cpu().numpy())
            actuals.extend(batch_y.cpu().numpy())

    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))[:, 0]
    actuals = scaler.inverse_transform(np.array(actuals).reshape(-1, 1))[:, 0]

    metrics = {
        'mse': mean_squared_error(actuals, predictions),
        'mae': mean_absolute_error(actuals, predictions),
        'r2': r2_score(actuals, predictions)
    }

    # Plot predictions vs actuals
    plt.figure(figsize=(12, 6))
    plt.scatter(actuals, predictions, alpha=0.5)
    plt.plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], 'r--')
    plt.xlabel('Actual Wind Speed')
    plt.ylabel('Predicted Wind Speed')
    plt.title('Predictions vs Actuals')
    plt.show()

    return metrics

In [ ]:
# Train and evaluate vanilla LSTM
vanilla_lstm = LSTM(
    input_size=CONFIG['input_size'],
    hidden_size=64,
    num_layers=2,
    dropout=0.2
)
vanilla_optimizer = torch.optim.Adam(vanilla_lstm.parameters(), lr=0.001)
criterion = nn.MSELoss()

vanilla_history = train_model(
    vanilla_lstm, train_loader, val_loader,
    criterion, vanilla_optimizer,
    CONFIG['device'], CONFIG['epochs']
)

In [ ]:
# Train and evaluate optimized LSTM
optimized_lstm = LSTM(
    input_size=CONFIG['input_size'],
    hidden_size=int(best_params['hidden_size']),
    num_layers=int(best_params['num_layers']),
    dropout=best_params['dropout']
)
optimized_optimizer = torch.optim.Adam(
    optimized_lstm.parameters(),
    lr=best_params['learning_rate']
)

optimized_history = train_model(
    optimized_lstm, train_loader, val_loader,
    criterion, optimized_optimizer,
    CONFIG['device'], CONFIG['epochs']
)

In [ ]:
# Compare and visualize results
print("\nVanilla LSTM Metrics:")
vanilla_metrics = evaluate_model(vanilla_lstm, val_loader, scaler, CONFIG['device'])
for metric, value in vanilla_metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nOptimized LSTM Metrics:")
optimized_metrics = evaluate_model(optimized_lstm, val_loader, scaler, CONFIG['device'])
for metric, value in optimized_metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
# Plot training history comparison
plt.figure(figsize=(12, 6))
plt.plot(vanilla_history['val_loss'], label='Vanilla LSTM', alpha=0.8)
plt.plot(optimized_history['val_loss'], label='Optimized LSTM', alpha=0.8)
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.title('Training History Comparison')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# %% [markdown]
# ## Save Models and Results

# %%
# Save models
def save_model(model, filename):
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': CONFIG,
        'scaler': scaler
    }, filename)

save_model(vanilla_lstm, 'vanilla_lstm.pth')
save_model(optimized_lstm, 'optimized_lstm.pth')


In [ ]:
# Save metrics comparison
results = {
    'vanilla_lstm': vanilla_metrics,
    'optimized_lstm': optimized_metrics,
    'best_parameters': best_params
}

# Convert to DataFrame for better visualization
results_df = pd.DataFrame({
    'Vanilla LSTM': vanilla_metrics,
    'Optimized LSTM': optimized_metrics
})

display(HTML(results_df.to_html()))

# Save results to CSV
results_df.to_csv('model_comparison_results.csv')

# %% [markdown]
# ## Prediction Function for New Data



In [ ]:
# %%
def predict_wind_speed(model, input_data, scaler, device=CONFIG['device']):
    """
    Make predictions for new input data

    Args:
        model: Trained LSTM model
        input_data: Input features (should match the expected input shape)
        scaler: Fitted scaler object
        device: Computing device (cuda/cpu)

    Returns:
        Predicted wind speed value
    """
    model.eval()
    with torch.no_grad():
        # Preprocess input
        scaled_input = scaler.transform(input_data)
        input_tensor = torch.FloatTensor(scaled_input).unsqueeze(0).to(device)

        # Get prediction
        output = model(input_tensor)

        # Inverse transform prediction
        prediction = scaler.inverse_transform(
            output.cpu().numpy().reshape(-1, 1)
        )[0, 0]

    return prediction

# Example usage:
"""
# For new data point:
new_data = np.array([[...]])  # Array with wind speeds at different heights
predicted_speed = predict_wind_speed(optimized_lstm, new_data, scaler)
print(f"Predicted wind speed at 107m: {predicted_speed:.2f} m/s")
"""

# %% [markdown]
# ## Conclusion and Model Analysis
#
# The notebook implements and compares two LSTM models for wind speed prediction:
# 1. Vanilla LSTM with standard hyperparameters
# 2. Optimized LSTM with Sparrow Search algorithm
#
# Key features implemented:
# - Batch normalization for improved training stability
# - Mixed precision training for better performance
# - Comprehensive evaluation metrics
# - Visualization of results
# - Model saving and loading functionality
#
# The optimized model shows [improvement/degradation] in performance compared to the vanilla model, particularly in terms of [specific metrics]